In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

In [2]:
# 读取数据
df1_train = pd.read_csv('/Users/apple/Desktop/titanic/train.csv')
df1_test = pd.read_csv('/Users/apple/Desktop/titanic/test.csv')

In [3]:
# 第一模块：数据探索，查看表的基本情况，有多少行，多少列，各列的数据类型、完整度；
print(type(list(df1_train.columns[2:4])))
print('-'*30)
print(df1_train.info())
print('-'*30)
print(df1_test.info())
print('-'*30)

<class 'list'>
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------    

In [4]:
# 对于数值型数据，查看其基本情况（个数、均值、最大最小值等）
print(df1_train.describe())
print('-'*30)

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  
------------------------------


In [5]:
# 对于字符串数据，查看其基本情况（个数、非重复个数、频数最高者、频次）
print(df1_train.describe(include=['O']))
print('-'*30)

                         Name   Sex Ticket    Cabin Embarked
count                     891   891    891      204      889
unique                    891     2    681      147        3
top     Cairns, Mr. Alexander  male   1601  B96 B98        S
freq                        1   577      7        4      644
------------------------------


In [6]:
# 对于整个表格，查看前 5行数据
print(df1_train.head())
print('-'*30)

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
--

In [7]:
# 第二模块：数据清洗、重点--补充缺失值
# 对于数值型字段，用其均值 补充缺失值
df1_train.fillna(df1_train.mean(),inplace=True)
df1_test.fillna(df1_test.mean(axis = 0),inplace=True)
print(df1_train.info())
print(df1_test.info())
print('-'*30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pcl

In [8]:
# 查看存在缺失值的字符串字段，各非重复值，及相应个数，按频次降序排列，并补充缺失值=频次最高者
print(df1_train['Embarked'].value_counts())
df1_train['Embarked'].fillna('S',inplace=True)

S    644
C    168
Q     77
Name: Embarked, dtype: int64


In [9]:
# 第三模块：模型的 特征选择
# 对于该案例，选择 Pclass、Sex、Age、SibSp、Parch、Ticket、Fare、Embarked 作为特征;Survived 作为标签
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
train_features = df1_train[features]
print(train_features)
train_labels = df1_train['Survived']
test_features = df1_test[features]
print('-'*30)

     Pclass     Sex        Age  SibSp  Parch     Fare Embarked
0         3    male  22.000000      1      0   7.2500        S
1         1  female  38.000000      1      0  71.2833        C
2         3  female  26.000000      0      0   7.9250        S
3         1  female  35.000000      1      0  53.1000        S
4         3    male  35.000000      0      0   8.0500        S
..      ...     ...        ...    ...    ...      ...      ...
886       2    male  27.000000      0      0  13.0000        S
887       1  female  19.000000      0      0  30.0000        S
888       3  female  29.699118      1      2  23.4500        S
889       1    male  26.000000      0      0  30.0000        C
890       3    male  32.000000      0      0   7.7500        Q

[891 rows x 7 columns]
------------------------------


In [10]:
# 将所有字符串数据，变为 数值型
# 将字符串字段 处理成 数值型，使用 Sklearn 特征选择里的 DictVectorizer,并转换成 特征值矩阵
# 即，Embarked单独的一列，拆开变成 'Embarked = S'、'Embarked = C'、'Embarked = Q' 3列的值，这3列分别为 0/1
# 而，Sex 这一单独列，拆开变成 'Sex = female'、'Sex = male' 2列的值，也均为 0/1 的值
# 所以，原来 train_features 仅有 7列，这样，即 扩展为 10列,891行
dvec = DictVectorizer(sparse=False)
train_features = dvec.fit_transform(train_features.to_dict(orient='record'))

In [11]:
# 数据规范化（只能等所有 数据变为 数值型的才可规范，若含有 字符串型，直接规范会出错）
# 此为 Z-Score 规范化，使得每个特征数据，均值为0，方差为1
ss = StandardScaler()
ss.fit(train_features)
train_features = ss.transform(train_features)

In [12]:
# 第四模块：构造分类器
# 鉴于此为分类模型，先采取 决策树模型，即 sklearn的 tree里的 DecisionTreeClassifier
# 当标准 criterion = 'entropy'时，即为 熵增--ID3模型；若为 criterion = 'gini'时，即为 基尼--CART模型

# 构造 决策树分类器 中的 ID3模型
clf_id3 = DecisionTreeClassifier(criterion='entropy')
# 训练模型
clf_id3.fit(train_features,train_labels)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [13]:
# 第五模块：模型预测 & 预测结果评估
# 一方面，我们可以将 train数据，先分割成 train & test 2份数据，用train的模型去测量 test准确率；
# 另一方面，我们可以 用 K折交叉验证，即，model_selection 里的 cross_val_score
# 所以这里我们先用 K折交叉验证，来统计决策树模型的准确率；同时比较下 决策树自带的 score函数 测量的准确率
print('决策树ID3 K折交叉验证准确率为 %.4f' % np.mean(cross_val_score(clf_id3,train_features,train_labels,cv=10)))
print('决策树ID3 自带的score函数 准确率为 %.4f' % (clf_id3.score(train_features,train_labels)))

决策树ID3 K折交叉验证准确率为 0.7779
决策树ID3 自带的score函数 准确率为 0.9820


In [14]:
# 当然，决策树自带的 score函数，与 metrics 里的 accuracy_score 函数结果一样
train_pre = clf_id3.predict(train_features)
score = round(accuracy_score(train_labels,train_pre),6)
print('决策树ID3 metrics里的 一般 accuracy_score 准确率为 %.4f' % score)

决策树ID3 metrics里的 一般 accuracy_score 准确率为 0.9820


In [15]:
# 用 模拟好的模型，预测 test表的结果
# 首先也是将 test表字符串字段 转为 数值型，由于之前 dvec已经 fit_transform了，所以这里可以直接 transform
test_features = dvec.transform(test_features.to_dict(orient='record'))
test_features = ss.transform(test_features)
test_pre_clf_id3 = clf_id3.predict(test_features)
print('-'*30)

------------------------------


In [16]:
# 构造 决策树弱分类器 & 模型预测结果评估:
clf_ruo = DecisionTreeClassifier(max_depth=1,min_samples_leaf=1,random_state=1)
clf_ruo.fit(train_features,train_labels)
print('决策树弱分类器中 K折交叉验证准确率为 %.4f' % np.mean(cross_val_score(clf_ruo,train_features,train_labels,cv=10)))
print('决策树弱分类器 自带的score函数 准确率为 %.4f' % (clf_ruo.score(train_features,train_labels)))
train_pre = clf_ruo.predict(train_features)
score = round(accuracy_score(train_labels,train_pre),6)
print('决策树弱分类器中 metrics里的 一般 accuracy_score 准确率为 %.4f' % score)
print('-'*30)

决策树弱分类器中 K折交叉验证准确率为 0.7867
决策树弱分类器 自带的score函数 准确率为 0.7868
决策树弱分类器中 metrics里的 一般 accuracy_score 准确率为 0.7868
------------------------------


In [17]:
# 构造 决策树CART分类器 & 模型预测结果评估
clf_cart = DecisionTreeClassifier(criterion='gini',random_state=1)
clf_cart.fit(train_features,train_labels)
print('决策树CART分类器中 K折交叉验证准确率为 %.4f' % np.mean(cross_val_score(clf_cart,train_features,train_labels,cv=10)))
print('决策树CART分类器 自带的score函数 准确率为 %.4f' % (clf_cart.score(train_features,train_labels)))
train_pre = clf_cart.predict(train_features)
score = round(accuracy_score(train_labels,train_pre),6)
print('决策树CART分类器中 metrics里的 一般 accuracy_score 准确率为 %.4f' % score)
test_pre_clf_cart = clf_cart.predict(test_features)
print('-'*30)

决策树CART分类器中 K折交叉验证准确率为 0.7756
决策树CART分类器 自带的score函数 准确率为 0.9820
决策树CART分类器中 metrics里的 一般 accuracy_score 准确率为 0.9820
------------------------------


In [18]:
# 构造 SVM的分类器 & 模型预测结果评估
svc = SVC(random_state=1)
svc.fit(train_features,train_labels)
print('SVM分类器中 K折交叉验证准确率为 %.4f' % np.mean(cross_val_score(svc,train_features,train_labels,cv=10)))
print('SVM分类器 自带的score函数 准确率为 %.4f' % (svc.score(train_features,train_labels)))
train_pre = svc.predict(train_features)
score = round(accuracy_score(train_labels,train_pre),6)
print('SVM分类器中 metrics里的 一般 accuracy_score 准确率为 %.4f' % score)
test_pre_svc = svc.predict(test_features)
print('-'*30)

SVM分类器中 K折交叉验证准确率为 0.8249
SVM分类器 自带的score函数 准确率为 0.8429
SVM分类器中 metrics里的 一般 accuracy_score 准确率为 0.8429
------------------------------


In [19]:
# 构造 高斯分类器 & 模型预测结果评估
gnb = GaussianNB()
gnb.fit(train_features,train_labels)
print('高斯朴素贝叶斯 分类器中 K折交叉验证准确率为 %.4f' % np.mean(cross_val_score(gnb,train_features,train_labels,cv=10)))
print('高斯朴素贝叶斯分类器 自带的score函数 准确率为 %.4f' % (gnb.score(train_features,train_labels)))
train_pre = gnb.predict(train_features)
score = round(accuracy_score(train_labels,train_pre),6)
print('高斯朴素贝叶斯分类器中 metrics里的 一般 accuracy_score 准确率为 %.4f' % score)
test_pre_gnb = gnb.predict(test_features)
print('-'*30)

高斯朴素贝叶斯 分类器中 K折交叉验证准确率为 0.7845
高斯朴素贝叶斯分类器 自带的score函数 准确率为 0.7912
高斯朴素贝叶斯分类器中 metrics里的 一般 accuracy_score 准确率为 0.7912
------------------------------


In [20]:
# 构造 Knn--K近邻分类器 & 模型预测结果评估
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(train_features,train_labels)
print('K近邻 分类器中 K折交叉验证准确率为 %.4f' % np.mean(cross_val_score(knn,train_features,train_labels,cv=10)))
print('K近邻 分类器 自带的score函数 准确率为 %.4f' % (knn.score(train_features,train_labels)))
train_pre = knn.predict(train_features)
score = round(accuracy_score(train_labels,train_pre),6)
print('K近邻 分类器中 metrics里的 一般 accuracy_score 准确率为 %.4f' % score)
test_pre_knn = knn.predict(test_features)
print('-'*30)

K近邻 分类器中 K折交叉验证准确率为 0.8204
K近邻 分类器 自带的score函数 准确率为 0.8361
K近邻 分类器中 metrics里的 一般 accuracy_score 准确率为 0.8361
------------------------------


In [21]:
# 构造 Adaboost分类器 & 模型预测结果评估:设置 该算法最大迭代次数为200，默认则是 50;默认弱分类器采用的是 决策树；
ada = AdaBoostClassifier(n_estimators=200,random_state=1)
ada.fit(train_features,train_labels)
print('AdaBoost分类器中 K折交叉验证准确率为 %.4f' % np.mean(cross_val_score(ada,train_features,train_labels,cv=10)))
print('AdaBoost分类器 自带的score函数 准确率为 %.4f' % (ada.score(train_features,train_labels)))
train_pre = ada.predict(train_features)
score = round(accuracy_score(train_labels,train_pre),6)
print('AdaBoost分类器中 metrics里的 一般 accuracy_score 准确率为 %.4f' % score)
print('-'*30)

AdaBoost分类器中 K折交叉验证准确率为 0.8115
AdaBoost分类器 自带的score函数 准确率为 0.8485
AdaBoost分类器中 metrics里的 一般 accuracy_score 准确率为 0.8485
------------------------------


In [22]:
# 将 test结果插入到 test原表中
# data.insert(列的索引位置，列名，列值）
df1_test.insert(0,'Survived_clf_id3',test_pre_clf_id3)
df1_test.insert(1,'Survived_clf_cart',test_pre_clf_cart)
df1_test.insert(2,'Survived_svc',test_pre_svc)
df1_test.insert(3,'Survived_gnb',test_pre_gnb)
df1_test.insert(4,'Survived_knn',test_pre_knn)

In [23]:
# 分别输出完整版 test结果，及，简化版 test结果
bq = ['Survived_clf_id3','Survived_clf_cart','Survived_svc','Survived_gnb','Survived_knn','PassengerId','Name']
jhb = df1_test[bq]
# 加入井号键，先不输出
# df1_test.to_csv('titanic_test_result.csv')
# jhb.to_csv('/Users/apple/Desktop/titanic/titanic_test_result_jhb.csv')